# INTRODUCTION

This kernel is meant to help solve one of this competition's initial chellenges - unification of all available datasets to be able to use all data for teaching our model

Here we will not be exploring data all that mutch, nor we will be engineering any new features. Instead we will unite all the data and see what kind of score we can get from this initial information using a few models

In this notebook we will perform the following operations:

- combine all available datasets to form complete applicants statistics 
- prepare data for modeling
    - impute missing values
    - scale data
- run baseline model - Logistic Regression
- run advanced model - XGBOOST
- compare results
- check our feature importance (as per our xgb model oppinion)

I will try to provide as detailed explaination as possible. 

**Any comments and or critisism are highly welcome =)**

In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

We can see that our data is rather big (hundreds of megabytes for some files). This may create problems when running heavy calculations, so below I have used some really cool formula I found to reduce the size of data being loaded. It basically iterates through all the data types within a file and substitutes data for a lightest possible format. I found it here : https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
       
        1. Iterate over every column
        2. Determine if the column is numeric
        3. Determine if the column can be represented by an integer
        4. Find the min and the max value
        5. Determine and apply the smallest datatype that can fit the range of values

    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

Now we can load in our data with this new function we've just created

In [3]:
train = import_data('../input/application_train.csv')

You can clearly see the data weight reduction while keeping all the information in place

Decreased by 79%

In [4]:
test = import_data('../input/application_test.csv')

In [5]:
bureau = import_data('../input/bureau.csv')

In [6]:
bureau_balance = import_data('../input/bureau_balance.csv')

In [7]:
previous_application = import_data('../input/previous_application.csv')

In [8]:
credit_card_balance = import_data('../input/credit_card_balance.csv')

In [9]:
installments_payments = import_data('../input/installments_payments.csv')

In [10]:
POS_CASH_balance = import_data('../input/POS_CASH_balance.csv')

# INITIAL EXPLORATION

In [11]:
train.TARGET.value_counts()

In [12]:
# we can see that our binary target variable is heavily disbalanced and is distributed at roughly 8%/92%
train.TARGET.plot.hist(color = 'g')

What about missing values?

In [13]:
# total percentage of NAs in data
sum(train.isna().sum())/(train.shape[0]*train.shape[1])*100

In [14]:
# now let's have a look which columns have what percentage of missing data (within train dataset only for now)

train_NA = (np.round((train.isna().sum()/len(train)),3)).sort_values(ascending = False)*100
train_NA = train_NA.sort_values(ascending = False)
train_NA.head(10)

Now let's have a look what data types do we have in our train set

In [15]:
train.dtypes.value_counts()

In [16]:
# what about subcategories within categorical data
train.select_dtypes('category').apply(pd.Series.nunique, axis = 0) # not so many

We will not cover visualizations and further exploratory analysis. Later on we can see all that we need within modeling and feature imporatnce matrix.

After all - data speaks for itself ,)

# UNITE ALL DATASETS

This is kinda tricky as data has different format and is represented differently

Our head dataset will be called 'data' which is just a merge of train and test

Other pieces we will add after some processing

**train + test**

this one is easy: both datasets have exactly the same format with only TARGET column being present in train set as the only difference

In [17]:
data = train.append(test)

In [18]:
# now just in case, let's check if we've got it right
data.TARGET.isna().sum() # same as number of test rows

In [19]:
sum(data.SK_ID_CURR[data.TARGET.isna()] == test.SK_ID_CURR) # all is good

In [20]:
sum(data.SK_ID_CURR.isin(test.SK_ID_CURR)) == len(test) # nothing else to prove

** + bureau**

Before we merge data with bureau, we need to merge bureau dataframe with related information in bureau_balance file

What is the exact problem here:
1. bureau dataframe comes from the Credit Bureau authority and displays one row for each credit the client from train/test dataset has taken previously. It is matched by SK_ID_CURR with train/test and where in train/test the SK_ID_CURR do not duplicate (1 for 1 client whom we are trying to classify) in most cases bureau dataframe has multipe indicies of the same client as he/she had applied to multiple loans previously.
2. in turn bureau_balance even more extends the previous credit information on a greater scale. It contains a separate row for each month of history of every previous credit reported to Credit Bureau (bureau dataframe) and is related to bureau df via SK_ID_BUREAU.

So the approach we are going to use is to calculate mean of each statistical column out of these both dataframes to include these mean values as features of our clients whom we are trying to classify. For example: mean days overdue for all credits that the client had previously taken.

I have to say that this approach leaves out some information such as categorical columns in some cases. For example the client with SK_ID_CURR = 666 had 7 credits in bureau dataframe, and when we collapse all these credits (grouped by one ID) into one line to indicate mean values for these credits, we will not be able to show a CREDIT_ACTIVE column that has different categorical values as Closed or Active for different previous credits. So this leaves room for some interesting feature engineering here.

Steps that we need to take:
1. Collapse bureau_balance dataframe to mean values grouped by SK_ID_BUREAU
2. Merge this with bureau dataframe
3. Collapse bureau dataframe to mean values grouped by SK_ID_CURR
4. Merge what we've got with our data df

Even though we've decided not to perform any feature engineering, one useful feature here is just asking for it. Let's calculate the total number of previous credits taken by each client and include this in our statistics. I believe that kind of information would be quite useful. So let's quickly do that before executing our program defined above

In [21]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'PREVIOUS_LOANS_COUNT'})
previous_loan_counts.head()

In [22]:
data = data.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')

Now back to merging with all the bureau and bureau_balance information

*STEP 1 - collapse bureau_balance*

In [23]:
# first define the formula for grouping rows by ID and calculating mean values
def extract_mean(x):
    y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_') 
    return y

In [24]:
# apply formula to create bureau_balance dataframe grouped by SK_ID_BUREAU with mean values of all numerical columns
bureau_bal_mean = extract_mean(bureau_balance)

In [25]:
bureau_bal_mean.head()

As you can see, this dataframe does not include the bureau_balance categorical column STATUS

Also note that our formula has changed the name of the SK_ID_BUREAU, we need to change it back in order to use it when merging with bureau df

One might argue that we didn't need to add this .add_prefix(...) to our formula above, but when working with larger datasets below it will prove itself useful

In [26]:
bureau_bal_mean = bureau_bal_mean.rename(columns = {'BUR_BAL_MEAN_SK_ID_BUREAU' : 'SK_ID_BUREAU'})

*STEP 2 - merge with bureau*

In [27]:
bureau = bureau.merge(bureau_bal_mean, on = 'SK_ID_BUREAU', how = 'left')
bureau.drop('SK_ID_BUREAU', axis = 1, inplace = True) # we don't need this internal ID anymore

*STEP 3 - collapse bureau*

In [28]:
def extract_mean(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_BUR_MEAN_') # note that we have changed the ID to group by and the prefix to add
    return y

In [29]:
bureau_mean_values = extract_mean(bureau)
bureau_mean_values = bureau_mean_values.rename(columns = {'PREV_BUR_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

In [30]:
bureau_mean_values.head(5)

Looks good. There are a few missing values although which we will deal with later

*STEP 4 - merge bureau with data*

In [31]:
data.shape

In [32]:
data = data.merge(bureau_mean_values, on = 'SK_ID_CURR', how = 'left')

In [33]:
data.shape

So here we've created 13 new features and added them to our train/test dataset called 'data'

**+ previous_application**

Quick information on this block of data: surprisingly... previous_application reflects clients' previous applications for loans to Home Credit.
As before, previous_application unfolds in a load of statistics with three other dataframes:
- POS_CASH_balance - monthly balance snapshots of previous Point Of Sale s and cash loans that the applicant had with Home Credit (one row for each month of history)
- installments_payments - repayment history for previous credits with Home Credit (one row for each payment)
- credit_card_balance - monthly balance snapshots of applicant's credit cards (one row for each month of history)

These 4 datasets have their own key to for internal mapping - SK_ID_PREV

So the plan here would be the following:

1. Collapse credit_card_balance dataframe to mean values grouped by SK_ID_PREV
2. Merge with previous_application (our 'leading' dataset in this case)
3. Collapse POS_CASH_balance to mean values grouped by SK_ID_PREV
4. Merge with previous_application
5. Collapse installments_payments ...
6. Merge with previous_application
7. Collapse the resulting previous_application dataset to mean values grouped by SK_ID_CURR
8. Merge our unfolded previous_application statistics with our data

But before we start, let's check if there are any records in previous_application that are not in our data?

In [34]:
len(previous_application.SK_ID_CURR.isin(data.SK_ID_CURR)) == len(previous_application)

looks good

One more thing! We will delete the SK_ID_CURR from the credit_card_balance / POS_CASH_balance / installment_payments as we do not need this column to be shown as mean, this information has no impact on statistics and will just clutter the space as noise. We will group them with our 'leading' dataset previous_application using SK_ID_PREV and our 'leading' dataset has this SK_ID_CURR key to be further mapped with our data

In [35]:
credit_card_balance.drop('SK_ID_CURR', axis = 1, inplace = True)
installments_payments.drop('SK_ID_CURR', axis = 1, inplace = True)
POS_CASH_balance.drop('SK_ID_CURR', axis = 1, inplace = True)

As previously, before tearing apart the previous_applications to Home Credit statistics, let's extract the number of previous applications of the clients to Home Credit and add this feature to our data

In [36]:
previous_application_counts = previous_application.groupby('SK_ID_CURR', as_index=False)['SK_ID_PREV'].count().rename(columns = {'SK_ID_PREV': 'PREVIOUS_APPLICATION_COUNT'})
previous_application_counts.head()

In [37]:
# and throw that column in our data
data = data.merge(previous_application_counts, on = 'SK_ID_CURR', how = 'left')

In [38]:
data.head(5)

Now back to our process

*STEP 1 - collapse credit_card_balance*

In [39]:
def extract_mean(x):
    y = x.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('CARD_MEAN_')
    return y

credit_card_balance_mean = extract_mean(credit_card_balance)

credit_card_balance_mean = credit_card_balance_mean.rename(columns = {'CARD_MEAN_SK_ID_PREV' : 'SK_ID_PREV'})

*STEP 2 - merge with previous_application*

In [40]:
previous_application = previous_application.merge(credit_card_balance_mean, on = 'SK_ID_PREV', how = 'left')

*STEP 3 - collapse installments_payments*

In [41]:
def extract_mean(x):
    y = x.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('INSTALL_MEAN_')
    return y

install_pay_mean = extract_mean(installments_payments)

install_pay_mean = install_pay_mean.rename(columns = {'INSTALL_MEAN_SK_ID_PREV' : 'SK_ID_PREV'})

*STEP 4 - merge with previous application*

In [42]:
previous_application = previous_application.merge(install_pay_mean, on = 'SK_ID_PREV', how = 'left')

*STEP 5 - collapse POS_CASH_balance*

In [43]:
def extract_mean(x):
    y = x.groupby('SK_ID_PREV', as_index=False).mean().add_prefix('POS_MEAN_')
    return y

POS_mean = extract_mean(POS_CASH_balance)

POS_mean = POS_mean.rename(columns = {'POS_MEAN_SK_ID_PREV' : 'SK_ID_PREV'})

*STEP 6 - merge with previous_application*

In [44]:
previous_application = previous_application.merge(POS_mean, on = 'SK_ID_PREV', how = 'left')

*STEP 7 - collapse the resulting previous_application dataset to show mean values grouped by SK_ID_CURR*

In [45]:
def extract_mean(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('PREV_APPL_MEAN_')
    return y

prev_appl_mean = extract_mean(previous_application)

prev_appl_mean = prev_appl_mean.rename(columns = {'PREV_APPL_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

prev_appl_mean = prev_appl_mean.drop('PREV_APPL_MEAN_SK_ID_PREV', axis = 1) # we don't need this intermediate column any more

*STEP 7 - merge what we've got with our data*

In [46]:
print('data shape', data.shape)
print('previous applications statistics shape', prev_appl_mean.shape)

In [47]:
data = data.merge(prev_appl_mean, on = 'SK_ID_CURR', how = 'left')

As we can see, this last sprint over previous applications added 50 new features to our statistics and completed the unification of all data

# PREPROCESS DATA

Up next:
1. Split for train and test datasets
2. One-hot encode categorical columns
3. Handle missing values
4. Scale data

**Split**

Perform split according to IDs in initial train and test datasets

In [48]:
train1 = data[data['SK_ID_CURR'].isin(train.SK_ID_CURR)]

test1 = data[data.SK_ID_CURR.isin(test.SK_ID_CURR)]
test1.drop('TARGET', axis = 1, inplace = True)

In [49]:
print('Training Features shape with categorical columns: ', train1.shape)
print('Testing Features shape with categorical columns: ', test1.shape)

**One-hot encode or create dummy variables**

What that is - is the representation of categorical columns that have two or more categories inside them as their own columns with binary numeric representation (0 or 1). For example in the initial dataframe you've got an EDUCATION variable which has subcategories as Primary/Middle/Higher/Phd/etc. Machine learning algorithms can not handle such data. So instead of storing all this information in one column called EDUCATION, we create 4 new separate columns according to subcategories and as a result for one client with Phd degree we will have 4 columns Primary / Middle / Higher / Phd. First three will display 0, as the applicant does not have Primary / Middle / Higher, and will display 1 in the Phd column. Thus our categorical column became numeric.

In [50]:
train1 = pd.get_dummies(train1)
test1 = pd.get_dummies(test1)

In [51]:
# After one-hot encoding it is usually a good idea to check the shapes again

print('Training Features shape with dummy variables: ', train1.shape)
print('Testing Features shape with dummy variables: ', test1.shape)

Looks like training data had more subcategories as the number of columns higher that in test set by 4. Actualy by 3, because we still have target in our train1 dataset

We can not have different features that categorize data in the training and testing datasets, so let's make them even

In [52]:
TARGET = train1.TARGET # save our TARGET variable
train1.drop('TARGET', axis = 1, inplace = True) # remove TARGET from train1

# allign the datasets
train1, test1 = train1.align(test1, join = 'inner', axis = 1)

In [53]:
print(train1.shape)
print(test1.shape)

**MISSING VALUES**

As a baseline model I've chosen logistic regression. For this algorithm in particular, and in general we need to deal with missing values. 

There are various methods such as pridicting the NAs based on other statistics, but we will use one of the simpler approaches and just fill in the mean or median values for the corresponding columns

In [54]:
# Define median imputation of missing values
imputer = Imputer(strategy = 'median')

In [55]:
print('Missing values in train data: ', sum(train1.isnull().sum()))
print('Missing values in test data: ', sum(test1.isnull().sum()))

In [56]:
# Fit imputer to our train dataset
imputer.fit(train1)

In [57]:
# Apply imputer to our dataframes
imputed_train = imputer.transform(train1)
imputed_test = imputer.transform(test1)

# the reason we created new objects (imputed_train/imputed_test) instead of just overriding train/test is because imputer creates 
    # arrays that include all the non missing values and imputed missing values, but it completely discards 
        # features names, which we will need later to examine feature importance

In [58]:
# now before we replace missing values by imputed, let's make sure we understand what will happen
train1.head(5)

Scroll right to APARTMENTS_AVG column to see the NaNs

In [59]:
# manually check what is the median of this column
train.APARTMENTS_AVG.median()

In [60]:
# replace the missing values with imputed ones
train1[train1.isnull()] = imputed_train
test1[test1.isnull()] = imputed_test

look again at this APARTMENTS_AVG column where those NaNs were and you can see that those missing values got replaced with median for this column that we had checked previously 0.087585

In [61]:
train1.head()

In [62]:
sum(train1.isnull().sum()) # we've gotten rid of all our NAs

**Scaling the data**

What is scaling? 
For example one person has a salary 100K, the other one, not as lucky, has 10K. Other parameter for example number of kids varies from 0 to 5. These two factors are in totally different orders: one being in hundreds of thousands other being in units. There are different scaling methods. The one I chose represents each column in values from 0 to 1 according to their relative values.

What is it for?
Basically, it creates less variance in data and smoothes it out to relative values making them easier to compare to each other.

In [63]:
# Define what type of scaling we will use 
scaler = MinMaxScaler(feature_range = (0, 1))

In [64]:
# Fit scaler to our training data
scaler.fit(train1)

In [65]:
# Calculate scaled values and store them in a separate object
scaled_train = scaler.transform(train1)
scaled_test = scaler.transform(test1)

In [66]:
# As before, in order to keep our column names we include scaled_values to our train/test dataframes like this

train1 = pd.DataFrame(scaled_train, index=train1.index, columns=train1.columns)
test1 = pd.DataFrame(scaled_test, index=test1.index, columns=test1.columns)

In [67]:
train1.head()

# MODELING

Basically we now have a dataset that includes all the features (not counting the droped categorical columns in bureau and previous_application data) all imputed and preprocessed

We can now try a few models to see the baseline result and compare it to some more advanced models as well as observe performance shifts introducing new features

**Baseline model - Logistic Regression**

In [68]:
# Define our model
GLM = LogisticRegression(C = 0.0001)

# Fit the model to the training data
GLM.fit(train1, TARGET)

# Predict on the testing data
GLM_pred = GLM.predict_proba(test1)[:, 1] # use predict_proba to predict probabilities and 
                                            # [:,1] for the 1 probability output (by default it will save both probabilities, 
                                                # for value 0 and value 1)
# Create the submission file
submission = test[['SK_ID_CURR']]
submission['TARGET'] = GLM_pred

In [69]:
submission.head()

In [70]:
# Save our submission to output
submission.to_csv('GLM_baseline.csv', index = False)

Result: 
- AUC 0.682
- Liederboard position 2100ish

If not for liederbord comparison, in real life we could not really tell wether this is good or bad

We can treat is a baseline that we should try to improve

**Advanced model - XGBOOST**

As an experiment we will run a more advanced model on the same data to see what could be the difference in prediction quality

In [71]:
# Give appropriate names to our key objects 
# (doesn't make any difference, but those are the correct names to be used not to get confused)

X_train = train1
y_train = TARGET

X_test = test1

# Create DMatrix objects (necessary for xgboost model)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

# Define model parameters
# These parameters are almost random, all of them really really should be 
    # tuned within cross validation, but that lies beyond our scope

params = {
    'min_child_weight': 19.0,         # minimum number of finally classified records in one leaf (the classifier will not break leaf further into pieces when the leaf size has reached 19 instances)
    'objective': 'binary:logistic',   # our objective is binary classificaiton
    'max_depth': 7,                   # maximum tree depth
    'eta': 0.025,                     # learning rate
    'eval_metric': 'auc',             # it is very important to train the model with a specific metric in mind, in our case - AUC
    'max_delta_step': 1.8,              # correct metric for train/valid is 20% success, because you can be doing a great job preparing data
    'colsample_bytree': 0.4,               # engineering features, etc, and teach the model to perform good at incorrect metric...
    'subsample': 0.8,
    'gamma': 0.65
    }

# This will verbose model training progress
watchlist = [(dtrain, 'train')]

XGB_model = xgb.train(params, dtrain, 
                300,                  # numrows
                watchlist,            # to see training performance (not really relevant without validaiton set)
                verbose_eval=50)      # to show training progress every 50 rounds

# Predict on the test data
XGB_pred = XGB_model.predict(dtest)

In [72]:
# Create submission file
submission_2 = test[['SK_ID_CURR']]
submission_2['TARGET'] = XGB_pred

In [73]:
submission_2.head()

In [74]:
# Export sumbmission
submission_2.to_csv('XGB_pred2.csv', index = False)

Result: 
- AUC 0.763
- Liederboard position 1500ish

A pretty impressive jump over roughly 600 positions up the ladder with just changing the model type

Althogh the result is far from perfect, this experiment demonstrated that xgboost is referred as "kaggle competition winning algorithm" for a reason

In [75]:
sns.set(font_scale = 1.2)
fig, ax = plt.subplots(figsize=(20, 40))

xgb.plot_importance(XGB_model, ax = ax)

Oups... I've forgotten that we've got 300+ features, which makes this plot cluttered no matter how large we make it

Let's extract importances values and construct our own plot with the top variables

In [76]:
importances = XGB_model.get_score()

importances = pd.DataFrame({'Importance': list(importances.values()), 'Feature': list(importances.keys())})
importances.sort_values(by = 'Importance', ascending = True,  inplace = True)
importances[importances['Importance'] > 200].plot(kind = 'barh', x = 'Feature', figsize = (8,12), color = 'orange')

This is a good alternative to initial data exploration. It gives us an unbias picture about which variables are actually important to the resulting prediction

# CONCLUSION

I believe that we have fulfilled the purpose of this notebook:

1. Combined all the given data
2. Performed basic minimum preprocessing
3. Created baseline model
4. Created advanced model to check the improvement potential
5. Created great basis for further improvement

What's next:
-	Inclusion of the (dropped during unification) categorical data (in numeric format)
-	Feature engineering. I can not stress this more than saying that extraction of new features is crucial for metrics improvement
-   Change missing values imputation strategy
-	Model tuning
    - Cross validation
    - Model ensembling

Your imagination is the limit…